# TP Deep Learning : Classification d’images avec TensorFlow/Keras
## Objectifs pédagogiques
- Appliquer les étapes d’un pipeline de classification d’images
- Comprendre et manipuler l’augmentation de données
- Construire un modèle inspiré de Xception
- Évaluer les performances d’un modèle
- Réaliser des prédictions personnalisées






##  1. Chargement des données

In [1]:
# Importation du framework TensorFlow et Keras pour construire le modèle de deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Importation de matplotlib pour l'affichage des images
import matplotlib.pyplot as plt

## 2. Montage de Google Drive

In [3]:
# Connexion à Google Drive pour accéder aux données
from google.colab import drive

# Montage du drive dans le répertoire '/content/drive'
drive.mount('/content/drive', force_remount=True)

/opt/anaconda3/envs/uevision/lib/python3.9/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


AttributeError: module 'IPython.utils.traitlets' has no attribute 'Unicode'

## 3. Définition du chemin vers le dossier de données

In [2]:
#Line paratge data  "https://drive.google.com/drive/folders/1SoohR88cc1sO1G3A30wxL4xnDcVUahOB?usp=drive_link"

# Définition du chemin vers le dossier contenant les images d'entraînement
traindir = '/Users/valentindaveau/Documents/UE_Vision/DATA/CAP/train'

# Affichage du contenu du dossier pour vérifier l'accès aux sous-dossiers/classes
!ls '/Users/valentindaveau/Documents/UE_Vision/DATA/CAP/train'


ok


## 4. Chargement des données sous forme de dataset TensorFlow

*  image_size = format unique pour toutes les images (200×200).




*  batch_size = combien d’images on donne au modèle en une fois.

*  train_ds = dataset d’entraînement (80% des images).

*  val_ds = dataset de validation (20% des images).

*  seed = permet de toujours avoir la même séparation (sinon ça change à chaque exécution).

In [3]:
# Définition de la taille des images d'entrée et de la taille du batch
image_size = (200,200)
batch_size = 32

In [4]:
# Chargement du dataset d'entraînement (80% des données)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    traindir,                 # Chemin vers les images
    validation_split=0.2,     # Fraction pour validation (20%)
    subset="training",        # Chargement de la partie entraînement
    seed=1337,                # Graine aléatoire pour réplication
    image_size=image_size,    # Redimensionnement des images
    batch_size=batch_size,    # Nombre d'images par batch
)

Found 9 files belonging to 1 classes.
Using 8 files for training.


In [ ]:
# Chargement du dataset de validation (20% des données)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    traindir,                 # Chemin vers les images
    validation_split=0.2,     # Même fraction pour validation
    subset="validation",      # Chargement de la partie validation
    seed=1337,                # Même graine pour cohérence
    image_size=image_size,    # Même taille d'image
    batch_size=batch_size,    # Même batch
)

##

## 5. Visualisation de quelques images du dataset


In [ ]:
# Création d'une figure de taille 10x10 pour afficher des exemples d'images
plt.figure(figsize=(10, 10))

# On prend un batch du dataset d'entraînement
for images, _ in train_ds.take(1):
    # Affichage des 6 premières images du batch




## 6. Augmentation des données


* RandomFlip → retourne les images horizontalement/verticalement

* RandomRotation → fait tourner les images

* RandomZoom → fait un zoom avant/arrière

* RandomCrop → découpe aléatoirement une partie de l’image

* RandomContrast → rend l’image plus claire ou plus sombre

* RandomSaturation → change l’intensité des couleurs

* RandomTranslation → déplace légèrement l’image en hauteur ou largeur

* GaussianNoise → ajoute un peu de bruit pour simuler des défauts capteur

In [ ]:
# Importation du module 'layers' depuis Keras
from tensorflow.keras import layers

# Définition d'un pipeline d'augmentation de données
data_augmentation = keras.Sequential(
    [
        #  Applique un retournement aléatoire horizontal ET vertical
        layers.RandomFlip("horizontal_and_vertical"),

        #  Applique une rotation aléatoire jusqu’à 20% de l'image
        #layers.RandomRotation(0.2),

        #  (À tester) Zoom aléatoire de 20% maximum
        # layers.RandomZoom(0.2),

        #  (À tester) Découpage/cadrage aléatoire de l’image
        # layers.RandomCrop(height=180, width=180),

        #  (À tester) Variation aléatoire de la luminosité/contraste
        # layers.RandomContrast(0.2),

        #  (À tester) Changement aléatoire de la saturation
        # layers.RandomSaturation(0.2),

        #  (À tester) Translation aléatoire (décalage en X ou Y)
        # layers.RandomTranslation(0.1, 0.1),

        #  (BONUS) Ajout de bruit pour simuler du bruit capteur
        # layers.GaussianNoise(0.1),
    ]
)


## Visualisation des images après augmentation

In [ ]:
# Création d'une figure de taille 10x10 pour afficher plusieurs images
plt.figure(figsize=(10, 10))

# On prend un seul batch d’images depuis le dataset d'entraînement
for images, _ in train_ds.take(1):

    # On affiche les 5 premières versions augmentées de la première image du batch
    for i in range(6):

        # Application du pipeline d'augmentation défini précédemment
        augmented_images = data_augmentation(images)

        # Création d’un sous-plot dans une grille 3x3 (position i+1)
        ax = plt.subplot(3, 3, i + 1)

        # Affichage de la première image augmentée sous forme d’image RGB
        plt.imshow(augmented_images[0].numpy().astype("uint8"))

        # On désactive les axes pour une meilleure lisibilité
        plt.axis("off")


## 7. Optimisation du pipeline avec prefetch()

Change la valeur de buffer_size (par ex. buffer_size=1 puis buffer_size=64).

Lance l’entraînement du modèle.*texte en italique*

Comparez le temps par époque → qu’observez-vous ?

In [ ]:
#  Préchargement des batches d'entraînement pendant que le GPU calcule déjà sur le batch courant
train_ds = train_ds.prefetch(buffer_size=32)

#  Préchargement des batches de validation de la même manière
val_ds = val_ds.prefetch(buffer_size=32)


## 8. Construction d’un modèle CNN

In [ ]:
def make_simple_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Bloc simple : convolution + pooling
    x = layers.Rescaling(1./255)(inputs)   # normalisation
    x = layers.Conv2D(32, 3, activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Flatten()(x)                # aplatissement
    x = layers.Dense(64, activation="relu")(x)

    if num_classes == 2:
        outputs = layers.Dense(1, activation="sigmoid")(x)
    else:
        outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs, outputs)

# Exemple : modèle simple pour des images 200x200 RGB
simple_model = make_simple_model(image_size + (3,), num_classes=2)
simple_model.summary()


In [ ]:
# Définition d'une fonction qui retourne un modèle Keras
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # ==== Bloc d'entrée ====
    # Data augmentation et normalisation
    x = data_augmentation(inputs)
    x = layers.Rescaling(1.0 / 255)(x)

    # ==== Bloc convolutionnel de base ====
    # Première convolution : motifs simples
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    # Deuxième convolution : motifs plus complexes
    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    # Sauvegarde pour la connexion résiduelle
    previous_block_activation = x

    # ==== Bloc convolutions profondes + résiduelles ====
    for size in [128, 256, 728]:
        # Convolutions séparables (plus efficaces)
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        # Réduction spatiale
        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Connexion résiduelle (chemin court)
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(previous_block_activation)
        x = layers.add([x, residual])
        previous_block_activation = x

    # ==== Bloc intermédiaire (8 répétitions) ====
    for _ in range(8):
        previous_block_activation = x

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(728, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(728, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(728, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        # Connexion résiduelle
        x = layers.Add()([x, previous_block_activation])
        previous_block_activation = x

    # ==== Bloc de sortie convolutif ====
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(728, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Réduction spatiale finale + connexion résiduelle
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    residual = layers.Conv2D(1024, 1, strides=2, padding="same")(previous_block_activation)
    x = layers.add([x, residual])
    previous_block_activation = x

    # Dernières convolutions
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(728, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # ==== Bloc final (classification) ====
    # Résumé global des features
    x = layers.GlobalAveragePooling2D()(x)

    # Dropout pour régularisation
    x = layers.Dropout(0.25)(x)

    # Couche dense de sortie
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    outputs = layers.Dense(units, activation=activation)(x)

    # Création du modèle final
    return keras.Model(inputs, outputs)




## Construction du modèle pour des images 200x200 avec 3 canaux (RGB)


In [ ]:
model = make_model(input_shape=image_size + (3,), num_classes=2)  # make_model  / make_simple_model

# Affichage du résumé graphique de l’architecture
keras.utils.plot_model(model, show_shapes=True)

## 9. Configuration dynamique des ressources (CPU & GPU)

CPU → sert à préparer et charger les images (data pipeline).

GPU → sert à entraîner le modèle (calcul matriciel très rapide).**texte en gras**

In [1]:
import sys, inspect, tensorflow as tf
print("Exec:", sys.executable)
print("TF:", tf.__version__)
print("TF file:", inspect.getfile(tf))
print("Built with GPU support:", tf.test.is_built_with_gpu_support())
print("Physical GPUs:", tf.config.list_physical_devices('GPU'))
print("Logical GPUs:", tf.config.list_logical_devices('GPU'))

NotFoundError: dlopen(/opt/anaconda3/envs/tfmetal39/lib/python3.9/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): symbol not found in flat namespace '_TF_GetInputPropertiesList'

In [ ]:

if tf.config.list_physical_devices('GPU'):
    with tf.device('/GPU:0'):
        a = tf.random.uniform((2048,2048))
        b = tf.random.uniform((2048,2048))
        c = tf.matmul(a,b)
    print("Matmul OK (GPU):", c.shape)
else:
    print("Pas de GPU détecté.")

Pas de GPU détecté.


In [ ]:
print("TF:", tf.__version__)
print("Built with GPU support:", tf.test.is_built_with_gpu_support())
print("Physical GPUs:", tf.config.list_physical_devices('GPU'))
print("Logical GPUs:", tf.config.list_logical_devices('GPU'))

TF: 2.12.0
Built with GPU support: False
Physical GPUs: []
Logical GPUs: []


In [2]:
# Importation du module multiprocessing pour détecter le nombre de cœurs CPU disponibles
import multiprocessing

# Définition d’une fonction pour estimer le nombre optimal de threads (CPU workers)
def set_workers(local=False):
    catcha = ''

    # Récupération du nombre total de cœurs CPU disponibles
    workers = multiprocessing.cpu_count()

    # Si on travaille en local, on en réserve 1 pour le système
    if local:
        workers -= 1
        catcha = 'locally'

    # Détection des GPU physiques disponibles avec TensorFlow
    gpus = tf.config.experimental.list_physical_devices('GPU')

    # Affichage des ressources disponibles
    print(f"Working with {workers} CPU threads {catcha} and with {len(gpus)} GPU")

    return workers

# Appel de la fonction pour stocker le nombre de workers dans une variable
workers_ = set_workers()


Working with 8 CPU threads  and with 0 GPU


## 10. Entraînement du modèle et suivi des performances

In [ ]:
# Définition du nombre d'époques d'entraînement
epochs = 3

# Définition des callbacks (fonctions appelées à chaque époque)
callbacks = [
    # Sauvegarde automatique du modèle à chaque époque dans un fichier .keras
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
# Compilation du modèle
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),  # Optimiseur Adam avec un taux d'apprentissage de 0.001
    loss="binary_crossentropy",             # Fonction de perte adaptée à la classification binaire
    metrics=["accuracy"],                   # Suivi de la précision pendant l'entraînement
)

In [ ]:

# Lancement de l'entraînement
history = model.fit(
    train_ds,               # Dataset d'entraînement
    epochs=epochs,          # Nombre d'époques
    callbacks=callbacks,    # Liste des callbacks à appliquer
    validation_data=val_ds, # Dataset de validation pour suivi du surapprentissage
)

# Affiche les métriques disponibles dans l'objet "history"
print(history.history.keys())

# Courbe de précision pour l'entraînement et la validation
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# Courbe de perte pour l'entraînement et la validation
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


## 11. Sauvegarde et rechargement du modèle

In [ ]:
# Sauvegarde du modèle au format .keras dans l’environnement local de Colab
model.save('modelCAPTEST.keras')

In [ ]:
# Sauvegarde du modèle dans Google Drive pour le conserver après la session
model.save('/content/drive/MyDrive/DATA/xxx')

In [ ]:
# Définition du chemin vers le modèle sauvegardé dans Drive
MODELdir = '/content/drive/MyDrive/DATA/xxxxxx'
# Chargement du modèle sauvegardé depuis Google Drive
model = keras.models.load_model(MODELdir)

In [3]:
# Définition du chemin vers le modèle sauvegardé dans Drive
MODELdir = '/Users/valentindaveau/Documents/UE_Vision/modelCAP3008HAMZA8TEST.keras'
# Chargement du modèle sauvegardé depuis Google Drive
model = keras.models.load_model(MODELdir)

TypeError: Could not deserialize class 'Functional' because its parent module keras.src.models.functional cannot be imported. Full object config: {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional_3', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 200, 200, 3], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer_1'}, 'registered_name': None, 'name': 'input_layer_1', 'inbound_nodes': []}, {'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 137532459331184}, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [32, 200, 200, 3], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer'}, 'registered_name': None}

## 12. Prédiction sur une image test

In [ ]:
import random
from glob import glob

# Lister toutes les images dans un dossier (exemple: classe NOK du test)
images_list = glob("/content/drive/MyDrive/DATA/CAP/test/nok/*.bmp")

# Sélection aléatoire
random_image_path = random.choice(images_list)
print("Image sélectionnée :", random_image_path)

# Charger et préparer l'image
img = keras.preprocessing.image.load_img(random_image_path, target_size=image_size)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # batch de 1

# Prédiction
pred = model.predict(img_array)
score = pred[0][0]

# Affichage
plt.imshow(img)
plt.title(f"Prediction : {'OK' if score>0.5 else 'NOK'} ({score:.2f})")
plt.axis("off")
plt.show()


## 13.  Prédictions groupées sur plusieurs images d’un dossier

In [ ]:
import random
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Récupération de toutes les images dans ton dossier
#  Modifie bien le chemin selon ta structure
images_list = glob("/content/drive/MyDrive/DATA/CAP/test/*/*.bmp")

# Si tu veux tester seulement un petit échantillon aléatoire (ex: 9 images)
images_list = random.sample(images_list, min(9, len(images_list)))

# Nombre d'images
n = len(images_list)

# Organisation de la grille
cols = 3
rows = (n // cols) + 1

plt.figure(figsize=(15, rows * 5))

for i, im in enumerate(images_list):
    ax = plt.subplot(rows, cols, i + 1)

    # Chargement + redimensionnement
    img = keras.preprocessing.image.load_img(im, target_size=image_size)

    # Conversion en tenseur utilisable par le modèle
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    # Prédiction
    predictions = model.predict(img_array, verbose=0)
    score = predictions[0][0]

    # Affichage
    plt.imshow(img)
    plt.title(f"{'OK' if score >= 0.5 else 'NOK'} ({score:.2f})")
    plt.axis("off")

plt.show()
